In [3]:
import numpy as np
from numpy.linalg import inv, det
from numpy import trace, abs, sqrt, zeros, pi


In [49]:
def py_matel(n, vechLk, vechLl, Sym, Mass, vecQ):
    
    # initialize arrays
    Lk=zeros((n,n));
    Ll=zeros((n,n));
    Ak=zeros((n,n));
    Al=zeros((n,n));
    Akl=zeros((n,n));
    invAkl=zeros((n,n));
    invAk=zeros((n,n));
    invAl=zeros((n,n));
    
    # build Lk and Ll

    #count=0;
    #for j=1:n
    #  for i=j:n
    #    count=count+1;
    #    Lk(i,j) = vechLk(count);
    #    Ll(i,j) = vechLl(count);
    #  end
    #end
    
    idxu = np.triu_indices(n);
    Lk[idxu] = vechLk;
    Ll[idxu] = vechLl;
    Lk=Lk.T
    Ll=Ll.T
    
    # apply symmetry projection on Ll
    
    Ll = Sym.T @ Ll;
    #Lk = Sym.T@Lk# @ Sym ;
    
    # build Ak, Al, Akl, invAkl, invAk, invAl

    Ak = Lk@Lk.T;
    Al = Ll@Ll.T;
    
    #Ak = (Sym.T@Ak@Sym)
    #Al = (Sym.T@Al@Sym)
    Akl = Ak+Al;
    
    print(det(Akl))
    
    invAkl = inv(Akl);
    invAk  = inv(Ak);
    invAl  = inv(Al);
    
    # Overlap: (normalized)
    skl = 2**(3*n/2) * sqrt( (abs(det(Lk))*abs(det(Ll))/det(Akl) )**3 );

    # kinetic energy

    tkl = skl*(6*trace(Mass@Ak@invAkl@Al));
    
    # potential energy
    
    RIJ = zeros((n,n));
    # 1/rij i~=j
    for j in range(0,n-1):
        for i in range(j+1,n):
            tmp2 = invAkl[i,i] + invAkl[j,j] - 2*invAkl[i,j];
            RIJ[i,j] = 2/sqrt(pi) * skl/sqrt(tmp2);


    # 1/rij i=j
    for i in range(0,n):
        RIJ[i,i] = 2/sqrt(pi) * skl/sqrt(invAkl[i,i]);
    
    Q =zeros((n,n))
    Q[idxu] = vecQ;
    Q=Q.T
    vkl = np.sum(RIJ*Q)

    hkl = tkl + vkl
    
    return {'skl':skl, 'tkl':tkl, 'vkl':vkl, 'hkl':hkl}

In [50]:
def test_matel():
    n = 3;
    vechLk = np.array([  1.00000039208682, 
              0.02548044275764261, 
              0.3525161612610669,
              1.6669144815242515,
              0.9630555318946559,
              1.8382882034659822 ]);
    
    vechLl = np.array([  1.3353550436464964,
               0.9153272033682132,
               0.7958636766525028,
               1.8326931436447955,
               0.3450426931160630,
               1.8711839323167831 ]);
    
    Sym = np.array([[0,0,1],
                    [0,1,0],
                    [1,0,0]]);
    
    Mass = np.array([[5.446170e-4, 2.723085077e-4, 2.723085077e-4],
                     [2.723085077e-4, .5002723085, 2.723085077e-4],
                     [2.723085077e-4, 2.723085077e-4, .5002723085 ]]);
    
    vecQ = [1, -1, -1, -1, 1, -1];
    
    matels = py_matel(n, vechLk, vechLl, Sym, Mass, vecQ)
    
    print('skl: ',matels['skl'])
    print('tkl: ',matels['tkl'])
    print('vkl: ',matels['vkl'])
    print('hkl: ',matels['hkl'])
    

In [4]:
test_matel()

skl:  0.5333557299037264
tkl:  4.3509068790883045
vkl:  -2.3839605567755693
hkl:  1.9669463223127353


In [22]:
test_matel()

170.69026506899922
skl:  0.5333557299037264
tkl:  4.3509068790883045
vkl:  -2.3839605567755693
hkl:  1.9669463223127353


In [51]:
test_matel()

170.69026506899922
skl:  0.5333557299037264
tkl:  4.3509068790883045
vkl:  -2.3839605567755693
hkl:  1.9669463223127353


In [52]:
Sym = np.array([[0,0,1],
                [0,1,0],
                [1,0,0]]);
print(Sym@Sym)
print(Sym.T@Sym)
print(det(Sym))

[[1 0 0]
 [0 1 0]
 [0 0 1]]
[[1 0 0]
 [0 1 0]
 [0 0 1]]
-1.0
